This notebook demonstrates how to integrate an existing LLM app with OTel tracing into TruLens. That is, we get both of:
1. Our TruLens spans will now work with the original OTel environment.
2. The spans from the original OTel environment will now be augmented to work with TruLens.

# App before TruLens

In [ ]:
# Create a (simple) LLM app that already has some OTel tracing in it.

import numpy as np


class MyApp:
    def orchestrator(self, prompt: str) -> float:
        if self.should_finish():
            with tracer.start_as_current_span("worked hard") as span:
                span.set_attribute("prompt", prompt)
            return self.get_final_score(prompt)
        res = self.do_research(prompt)
        return self.orchestrator(res)

    def should_finish(self) -> bool:
        return np.random.random() < 0.25

    def get_final_score(self, prompt: str) -> float:
        return np.random.random()

    def do_research(self, prompt: str) -> str:
        return prompt + "\ndoing hard work..."

In [ ]:
# Set up OTel tracing.

from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import ConsoleSpanExporter
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

tracer_provider = TracerProvider()
exporter = ConsoleSpanExporter(
    formatter=lambda span: f"span name: {span.name}\n\tattributes: {span.attributes}\n\n"
)
span_processor = SimpleSpanProcessor(exporter)
tracer_provider.add_span_processor(span_processor)
trace.set_tracer_provider(tracer_provider)
tracer = tracer_provider.get_tracer(__name__)

In [ ]:
# Call app.

import time

app = MyApp()
app.orchestrator("Hello, world!")

tracer_provider.force_flush()

# Allow time to flush and print OTel spans to stdout.
time.sleep(1)

# Integration into TruLens

In [ ]:
# Ensure before any TruLens packages are imported this environment variable is
# set.

import os

os.environ["TRULENS_OTEL_TRACING"] = "1"

In [ ]:
# Add TruLens instrumentation to the app. The only difference here is that we
# have added `@instrument` decorators. In reality, you would not do this, but
# just add the `@instrument` decorators to the MyApp class above.


from trulens.core.otel.instrument import instrument


class MyAppWithTruLensInstrumentation:
    @instrument()
    def orchestrator(self, prompt: str) -> float:
        if self.should_finish():
            with tracer.start_as_current_span("worked hard") as span:
                span.set_attribute("prompt", prompt)
            return self.get_final_score(prompt)
        res = self.do_research(prompt)
        return self.orchestrator(res)

    @instrument()
    def should_finish(self) -> bool:
        return np.random.random() < 0.25

    @instrument()
    def get_final_score(self, prompt: str) -> float:
        return np.random.random()

    @instrument()
    def do_research(self, prompt: str) -> str:
        return prompt + "\ndoing hard work..."

In [ ]:
# Create the TruSession and TruApp.

from trulens.apps.app import TruApp
from trulens.core import TruSession

tru_session = TruSession()  # By default, this will use a local SQLite database.
tru_session.reset_database()

app_with_trulens_instrumentation = MyAppWithTruLensInstrumentation()
tru_app = TruApp(
    app=app_with_trulens_instrumentation,
    app_name="My App",
    app_version="1.0.0",
)

In [ ]:
# Invoke app this time with TruLens. Notice that the original non-TruLens OTel
# exporter now sees TruLens spans.

import time

with tru_app:
    app_with_trulens_instrumentation.orchestrator("Hello, TruLens!")

tru_session.force_flush()

# Allow time to flush and print OTel spans to stdout.
time.sleep(1)

In [ ]:
# View the TruLens spans in the database (and note the original "worked hard"
# non-TruLens span is still there).

import pandas as pd
import sqlalchemy as sa

db = tru_session.connector.db
with db.session.begin() as db_session:
    q = sa.select(db.orm.Event).order_by(db.orm.Event.start_timestamp)
    df = pd.read_sql(q, db_session.bind)
df["name"] = df["record"].apply(lambda x: x["name"])
df["attributes"] = df["record_attributes"]
df = df[["name", "attributes"]]
df